# import libs


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter, LabelModel


# populates database


In [27]:
base = 'database.txt'
data = pd.read_csv(base, sep='\t', encoding='utf-8',
                   names=['text'], header=None)


In [28]:
data


,text
0,'@galliyanamiteva kkkkkkkkkk eu sou bue caseir...
1,"'Covid-19: Brasil tem 28,7 milhões de casos e ..."
2,'A mesma globo que recentemente defendia o iso...
3,'Covid era altura de fazeres a tua parte e aju...
4,'Feriadão prolongado desse e não vou poder ver...
...,...
12995,'@freeblowjob Eu fiquei um tempo sem falar com...
12996,'Vacinação em Coimbra passa a ser feita nos ce...
12997,'RiRi não para mais em casa? cê tá grávida e a...
12998,'Gilmar Mendes pediu para apagar todos os arqu...


# create labels


In [29]:
USEFULL = 1
USELESS = -1
ABSTAIN = 0


# creates train and test sets


In [30]:
df_train, df_test = train_test_split(data, train_size=0.8)


In [31]:
df_train


,text
3809,"'Eu tô gripada pra caramba, pode até se covid ..."
2306,'Covid-19: estudo mostra sucesso de inibidores...
8684,'Esse ano eu ia curtir Carnaval pq finalmente ...
2374,'🗞️ Secretaria de Saúde orienta cuidados contr...
9234,"'Brasil, 25 de fevereiro de 2022.\nPaciente ac..."
...,...
7763,"'Semaninha complicada , até achei que estava c..."
5218,'Na real q foi assim*\n\nPré escola = guerra\n...
12252,'@rafaelatsh A cabeça pós covid \U0001f972'
1346,'@chrisgonzatti Sem Bolsonaro. Sem Covid. Sem ...


# creates label functions


In [32]:
@labeling_function()
def label1(x):
    return USEFULL if "vacina" in x.text.lower() else ABSTAIN


@labeling_function()
def label2(x):
    return USEFULL if "bolsonaro" in x.text.lower() else ABSTAIN


@labeling_function()
def label3(x):
    return USEFULL if "pandemia" in x.text.lower() else ABSTAIN


@labeling_function()
def label4(x):
    return USEFULL if "corona" in x.text.lower() else ABSTAIN


@labeling_function()
def label5(x):
    return USEFULL if "covid" in x.text.lower() else ABSTAIN


@labeling_function()
def label6(x):
    return USEFULL if "virus" in x.text.lower() else ABSTAIN


@labeling_function()
def label7(x):
    return USELESS if "caralho" in x.text.lower() else ABSTAIN


@labeling_function()
def label8(x):
    return USELESS if "puta" in x.text.lower() else ABSTAIN


@labeling_function()
def label9(x):
    return USELESS if "merda" in x.text.lower() else ABSTAIN


@labeling_function()
def label10(x):
    return USELESS if "porra" in x.text.lower() else ABSTAIN


# agroups label functions


In [33]:
lfs = [label1, label2, label3, label4, label5,
       label6, label7, label8, label9, label10]


# applies label functions


In [34]:
applier = PandasLFApplier(lfs=lfs)


In [35]:
L_train = applier.apply(df=df_train)


100%|██████████| 10400/10400 [00:01<00:00, 8373.39it/s]


In [36]:
L_train


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# analyzes label functions


In [37]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()


,j,Polarity,Coverage,Overlaps,Conflicts
label1,0,"[0, 1]",1.000000,1.000000,1.000000
label2,1,"[0, 1]",1.000000,1.000000,1.000000
label3,2,"[0, 1]",1.000000,1.000000,1.000000
label4,3,"[0, 1]",1.000000,1.000000,1.000000
label5,4,"[0, 1]",1.000000,1.000000,1.000000
label6,5,"[0, 1]",1.000000,1.000000,1.000000
label7,6,[0],0.992885,0.992885,0.992885
label8,7,[0],0.989519,0.989519,0.989519
label9,8,[0],0.988077,0.988077,0.988077
label10,9,[0],0.989712,0.989712,0.989712


# labels model

In [38]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)


INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=57.715]
INFO:root:[100 epochs]: TRAIN:[loss=0.012]
 57%|█████▋    | 284/500 [00:00<00:00, 1432.67epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.002]
INFO:root:[400 epochs]: TRAIN:[loss=0.001]
100%|██████████| 500/500 [00:00<00:00, 1396.88epoch/s]
INFO:root:Finished Training


In [39]:
df_train.head()


,text
3809,"'Eu tô gripada pra caramba, pode até se covid ..."
2306,'Covid-19: estudo mostra sucesso de inibidores...
8684,'Esse ano eu ia curtir Carnaval pq finalmente ...
2374,'🗞️ Secretaria de Saúde orienta cuidados contr...
9234,"'Brasil, 25 de fevereiro de 2022.\nPaciente ac..."


In [40]:
df_train['Labels'] = label_model.predict(L=L_train, tie_break_policy="abstain")


/var/folders/47/2r15btws4hl473544pb4_s700000gp/T/ipykernel_37396/402419057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Labels'] = label_model.predict(L=L_train, tie_break_policy="abstain")


In [41]:
df_train.query('Labels == 0')


,text,Labels
3809,"'Eu tô gripada pra caramba, pode até se covid ...",0
2306,'Covid-19: estudo mostra sucesso de inibidores...,0
8684,'Esse ano eu ia curtir Carnaval pq finalmente ...,0
2374,'🗞️ Secretaria de Saúde orienta cuidados contr...,0
9234,"'Brasil, 25 de fevereiro de 2022.\nPaciente ac...",0
...,...,...
7763,"'Semaninha complicada , até achei que estava c...",0
5218,'Na real q foi assim*\n\nPré escola = guerra\n...,0
12252,'@rafaelatsh A cabeça pós covid \U0001f972',0
1346,'@chrisgonzatti Sem Bolsonaro. Sem Covid. Sem ...,0
